# ColumnStoreOptimize

## 1.0 Introduction

Most Fact tables within a synapse dimensional data warehouse should be using a clustered column store. This will compress data using a column store into segments of about a million rows, across the 60 distributions. Hence a common recommendation is that tables below 60 million rows don’t get as much benefit from column stores in SQL Pools, whereas they may on SMP platform such as SQLDB or IaaS.

If a data is only ever loaded once into a column store index and there is no special tuning such as ordering (discussed later), then there is pretty much no maintenance to be done.

If data is gradually inserted/deleted/updated, then we can be left with “delta stores” which are a row store contained within the column store containing new rows and “soft deletes”. These can hurt performance in two ways

- The effective size of a column store becomes much smaller than the 1 million expected
- The query has to traverse both the columns store and then the delta store to contra out deletes and add in uncompressed/open rows.

## 1.1 Locating ColumnStores that need maintenance

We have three scenarios that may class a column store as requiring maintenance.

- The Avg Row size of a segment is below the optimal (1024\*1024=1,048,576). We call this density and it can be expressed as %. Eg 10% fragmentation means each segment is only 90% full.
- The table contains a lot of soft deleted rows
- The table contains a lot of inserted rows that have not yet been compressed by the [tuple mover](https://docs.microsoft.com/en-us/azure/synapse-analytics/sql-data-warehouse/sql-data-warehouse-tables-index). It can take over 60 million rows before this is engaged (eg 1 segment per distribution).

### Demo - Fragmentation and Density

In [11]:
/* Demo - Examining Column store Density on inital Insert fro 100 milion row */
IF OBJECT_ID('[dbo].[FactFinance100m]') is not null 
	DROP TABLE [dbo].[FactFinance100m]
GO
CREATE TABLE [dbo].[FactFinance100m] WITH (
	DISTRIBUTION = ROUND_ROBIN, CLUSTERED COLUMNSTORE INDEX 
) AS
SELECT TOP 130000000 * FROM [dbo].[FactFinance1b]



Commands completed successfully.

(130000000 rows affected)

Total execution time: 00:08:09.671

In [12]:

/* This view shows the fragmentation_density is perfect, with some open row stores for overspill. Not ununsual as this table is only 2 complete segments per distribution */ 
SELECT * FROM dbo.vColumnstoreStats WHERE table_name='FactFinance100m'



(1 row affected)

Total execution time: 00:00:02.870

execution_date,database_name,schema_name,table_name,partition_number,partition_scheme,object_id,index_name,row_count,deleted_row_count,row_group_count,compressed_row_count,compressed_rowgroup_count,open_rowgroup_count,open_row_count,compressed_row_max,compressed_row_avg,fragmentation_density,fragmentation_deletes,fragmentation_open
2021-08-22 18:17:47.190,AdventureWorksDW,dbo,FactFinance100m,NULL,NULL,1924917929,ClusteredIndex_90da849ca6c24fafbf608f90686201ca,130000000,0,180,125829120,120,60,4170880,1048576,1048576,0.0000,0.0000,6.6300


In [14]:
/* Using the DMV sys.[dm_pdw_nodes_db_column_store_row_group_physical_stats] we can track the size of each segment and WHY it was closed 
    The view "[dbo].[vCS_rg_physical_stats]" from https://docs.microsoft.com/en-us/azure/synapse-analytics/sql-data-warehouse/sql-data-warehouse-memory-optimizations-for-columnstore-compression 
    Helps a lot in understanding why segments arent full.

    BULK INSERT, over partitioning or over enthusiastic REORG are the.

    In this case we see only OPEN and NO_TRIM

*/
select * From [dbo].[vCS_rg_physical_stats] WHERE logical_table_name='FactFinance100m'

(180 rows affected)

Total execution time: 00:00:02.212

logical_table_name,row_group_id,partition_number,state,state_desc,total_rows,trim_reason_desc,physical_name,created_time
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_1,2021-08-22 18:14:29.690
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_2,2021-08-22 18:15:18.770
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_3,2021-08-22 18:15:09.563
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_4,2021-08-22 18:15:18.877
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_5,2021-08-22 18:14:11.883
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_6,2021-08-22 18:15:09.660
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_7,2021-08-22 18:15:06.670
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_8,2021-08-22 18:14:29.497
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_9,2021-08-22 18:15:06.697
FactFinance100m,2,1,1,OPEN,69376,NULL,Table_3f728d8864434c88a9f60907a5c2168f_10,2021-08-22 18:14:29.970


In [15]:
/*
    Lets create same table, but just with 20 million rows (not complete segment, then insert in batches to make 130 million)

    Here we can see the RowGroup closed prematurely by BULOAD. If BULK operation si > about 200k records a close may happen on inital insert.

    This results in 14% fragmentation and veery small RowGroupo
*/

IF OBJECT_ID('[dbo].[FactFinance100m]') is not null 
	DROP TABLE [dbo].[FactFinance100m]
GO
CREATE TABLE [dbo].[FactFinance100m] WITH (
	DISTRIBUTION = ROUND_ROBIN, CLUSTERED COLUMNSTORE INDEX 
) AS
SELECT TOP 20000000 * FROM [dbo].[FactFinance1b]
GO
SELECT * FROM dbo.vColumnstoreStats WHERE table_name='FactFinance100m'
select * From [dbo].[vCS_rg_physical_stats] WHERE logical_table_name='FactFinance100m'


Commands completed successfully.

(20000000 rows affected)

(1 row affected)

(60 rows affected)

Total execution time: 00:03:12.995

execution_date,database_name,schema_name,table_name,partition_number,partition_scheme,object_id,index_name,row_count,deleted_row_count,row_group_count,compressed_row_count,compressed_rowgroup_count,open_rowgroup_count,open_row_count,compressed_row_max,compressed_row_avg,fragmentation_density,fragmentation_deletes,fragmentation_open
2021-08-22 18:37:43.140,AdventureWorksDW,dbo,FactFinance100m,NULL,NULL,1940917986,ClusteredIndex_1fba0db5c48b40288124497ec2198389,20000000,0,60,20000000,60,0,0,333808,333333,68.2100,0.0000,0.0000


logical_table_name,row_group_id,partition_number,state,state_desc,total_rows,trim_reason_desc,physical_name,created_time
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_1,2021-08-22 18:37:35.767
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_2,2021-08-22 18:37:38.800
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_3,2021-08-22 18:37:38.783
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_4,2021-08-22 18:37:35.057
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_5,2021-08-22 18:37:40.100
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_6,2021-08-22 18:37:41.600
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_7,2021-08-22 18:37:41.603
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_8,2021-08-22 18:37:35.160
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_9,2021-08-22 18:37:40.050
FactFinance100m,0,1,3,COMPRESSED,333312,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_10,2021-08-22 18:37:34.487


In [16]:
/*
    Lets Insert another 100 million to make similar as inital test case (120 million)

    We can see that we have 360 RowGroups instead of 180 and Fragmentation is only 17%, with only 33k per Row Group instead of 1024l
*/

INSERT INTO FactFinance100m ([AccountKey], [ScenarioKey], [DepartmentGroupKey], [DateKey], [OrganizationKey], [Amount], [Date], [LineageKey])
SELECT TOP 20000000 [AccountKey], [ScenarioKey], [DepartmentGroupKey], [DateKey], [OrganizationKey], [Amount], [Date], [LineageKey] FROM [dbo].[FactFinance1b]
GO 5


Beginning execution loop

(20000000 rows affected)

(20000000 rows affected)

In [18]:
/* Now lets Look at the Fragmnentation. 68% Fragmented. Eg only 32% full */

SELECT * FROM dbo.vColumnstoreStats WHERE table_name='FactFinance100m'
select * From [dbo].[vCS_rg_physical_stats] WHERE logical_table_name='FactFinance100m'

(1 row affected)

(180 rows affected)

Total execution time: 00:00:04.837

execution_date,database_name,schema_name,table_name,partition_number,partition_scheme,object_id,index_name,row_count,deleted_row_count,row_group_count,compressed_row_count,compressed_rowgroup_count,open_rowgroup_count,open_row_count,compressed_row_max,compressed_row_avg,fragmentation_density,fragmentation_deletes,fragmentation_open
2021-08-22 18:45:21.170,AdventureWorksDW,dbo,FactFinance100m,NULL,NULL,1940917986,ClusteredIndex_1fba0db5c48b40288124497ec2198389,60000000,0,180,60000000,180,0,0,333884,333333,68.2100,0.0000,0.0000


logical_table_name,row_group_id,partition_number,state,state_desc,total_rows,trim_reason_desc,physical_name,created_time
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_1,2021-08-22 18:44:29.980
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_2,2021-08-22 18:44:30.760
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_3,2021-08-22 18:44:31.943
FactFinance100m,2,1,3,COMPRESSED,333884,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_4,2021-08-22 18:44:27.803
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_5,2021-08-22 18:44:30.640
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_6,2021-08-22 18:44:30.880
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_7,2021-08-22 18:44:29.987
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_8,2021-08-22 18:44:20.310
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_9,2021-08-22 18:44:27.783
FactFinance100m,2,1,3,COMPRESSED,333324,BULKLOAD,Table_8c92dbd39040490a8eae0c6877992248_10,2021-08-22 18:44:30.793


In [20]:
/* We could fix this with a REORG 
    ALTER INDEX ClusteredIndex_1fba0db5c48b40288124497ec2198389 ON [dbo].[FactFinance100m] REORGANIZE

    But, lets introduce the ColumnStoreOptimize which can locate all Column Stores with low density and REORG them
*/
exec   [dbo].[ColumnstoreOptimize]  @Tables='FactFinance100m'
,@DensityThreshold=25 /* Default */
,@OPenThreshold=null
,@DeleteThreshold=null
,@TimeLimit =null
, @Execute='Y'


Date and time: 2021-08-22 18:49:47

Server: aw-dev

Version: 10.0.13582.0

Edition: SQL Azure

Parameters: @Tables= 'FactFinance100m', @DensityThreshold = 25.00, @OpenThreshold = 200000, @DeleteThreshold = 200000, @Execute = Y

...

Date and time: 2021-08-22 18:49:52

[dbo].[FactFinance100m]

SqlCommand: ALTER INDEX ClusteredIndex_1fba0db5c48b40288124497ec2198389 ON [dbo].[FactFinance100m] REORGANIZE

180 60000000 0 0 68.21%

Total execution time: 00:00:32.418

In [21]:
/*  Now we can check commandlog and also imprves Stats
    From 180 row groups to just 60 (one per distibution)
 */
SELECT TOP 1 * FROM dbo.CommandLog ORDER BY StartTime DESC
SELECT * From vColumnstoreStats where table_name ='factFinance100m'


(1 row affected)

(1 row affected)

Total execution time: 00:00:02.502

ID,DatabaseName,SchemaName,ObjectName,ObjectType,IndexName,IndexType,StatisticsName,PartitionNumber,ExtendedInfo,Command,CommandType,StartTime,EndTime,ErrorNumber,ErrorMessage
396,AdventureWorksDW,dbo,FactFinance100m,U,ClusteredIndex_1fba0db5c48b40288124497ec2198389,5,NULL,NULL,<ExtendedInfo><RowGroups>180</RowGroups><Rows>60000000</Rows><OpenRows>0</OpenRows><DeletedRows>0</DeletedRows><DensityFragmentation>68.21%</DensityFragmentation></ExtendedInfo>,ALTER INDEX ClusteredIndex_1fba0db5c48b40288124497ec2198389 ON [dbo].[FactFinance100m] REORGANIZE,ALTER INDEX,2021-08-22 18:49:52.140,2021-08-22 18:50:17.373,NULL,NULL


execution_date,database_name,schema_name,table_name,partition_number,partition_scheme,object_id,index_name,row_count,deleted_row_count,row_group_count,compressed_row_count,compressed_rowgroup_count,open_rowgroup_count,open_row_count,compressed_row_max,compressed_row_avg,fragmentation_density,fragmentation_deletes,fragmentation_open
2021-08-22 18:52:13.123,AdventureWorksDW,dbo,FactFinance100m,NULL,NULL,1940917986,ClusteredIndex_1fba0db5c48b40288124497ec2198389,60000000,0,60,60000000,60,0,0,1000520,1000000,4.6300,0.0000,0.0000
